#### System Design 
- Marcelo Hernandez 
- August 2024

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import joblib

In [2]:
# Constants
MAX_HEATING_PER_DAY = 2
HEATING_DURATION = timedelta(hours=4)
VENTILATION_DURATION = timedelta(minutes=15)
SATISFACTION_THRESHOLD = 100  # So that we're always optimising for satisfaction
CO2_CONSTANT = 194.4319985  # Assumption for simulation simplification

In [3]:
# Load datasets and model
new_day_df = pd.read_csv('../data/New_day_with_satisfactions.csv')
training_df = pd.read_csv('../data//6com2007-3_cleaned.csv')
trained_model = joblib.load('../src/random_forest_model.pkl')
scaler = joblib.load('../src/scaler.pkl')

print(new_day_df)
print(training_df)

          Date   Time    CO2_room  Relative_humidity_room  Lighting_room  \
0   12/04/2012  00:00  198.967871               53.699924      13.814564   
1   12/04/2012  00:15  198.618986               52.509405      13.131954   
2   12/04/2012  00:30  198.828400               51.386424      14.304592   
3   12/04/2012  00:45  198.422066               54.805172      13.680236   
4   12/04/2012  01:00  198.931140               52.759482      13.547493   
..         ...    ...         ...                     ...            ...   
91  12/04/2012  22:45  193.083867               35.551432      13.339227   
92  12/04/2012  23:00  193.211850               37.567429      12.856611   
93  12/04/2012  23:15  192.572170               36.010513      14.207420   
94  12/04/2012  23:30  192.037088               36.027294      14.026930   
95  12/04/2012  23:45  192.062446               39.082082      13.853875   

    Meteo_Rain  Meteo_Sun_dusk  Meteo_Wind  Meteo_Sun_light_in_west_facade  \
0        

In [4]:
# Get outdoor conditions for ventilation
def get_outdoor_conditions(timestamp, df):
    time_str = timestamp.strftime('%H:%M')
    conditions = df[df['Time'] == time_str]
    return conditions['Outside temp'].mean(), conditions['Humidity'].mean()

# Test for all values with 11:45 timestamp
timestamp = datetime.strptime('11:45:00', '%H:%M:%S')
outdoor_temp, outdoor_humidity = get_outdoor_conditions(timestamp, training_df)
print(outdoor_temp, outdoor_humidity)

19.086896551724138 46.096741379310345


In [5]:
# Simulation time setup
simulation_start = '13/03/2012 11:45'
simulation_end = '11/04/2012 06:30'
start_datetime = datetime.strptime(simulation_start, '%d/%m/%Y %H:%M')
end_datetime = datetime.strptime(simulation_end, '%d/%m/%Y %H:%M')

print("Start:", start_datetime, "-> End:", end_datetime)

# Simulation DataFrame
simulation_df = pd.DataFrame(index=pd.date_range(start=start_datetime, end=end_datetime, freq='15T'),
                             columns=['Indoor Temperature', 'Indoor Humidity', 'CO2 Levels', 'Predicted Satisfaction'])

print("Simulation DataFrame")
print(simulation_df)

Start: 2012-03-13 11:45:00 -> End: 2012-04-11 06:30:00
Simulation DataFrame
                    Indoor Temperature Indoor Humidity CO2 Levels  \
2012-03-13 11:45:00                NaN             NaN        NaN   
2012-03-13 12:00:00                NaN             NaN        NaN   
2012-03-13 12:15:00                NaN             NaN        NaN   
2012-03-13 12:30:00                NaN             NaN        NaN   
2012-03-13 12:45:00                NaN             NaN        NaN   
...                                ...             ...        ...   
2012-04-11 05:30:00                NaN             NaN        NaN   
2012-04-11 05:45:00                NaN             NaN        NaN   
2012-04-11 06:00:00                NaN             NaN        NaN   
2012-04-11 06:15:00                NaN             NaN        NaN   
2012-04-11 06:30:00                NaN             NaN        NaN   

                    Predicted Satisfaction  
2012-03-13 11:45:00                    NaN  
2012-

In [6]:
# Initialise with the first known conditions at 11:45
initial_conditions = training_df[training_df['Time'] == '11:45']
initial_temp = initial_conditions['Indoor_temperature_room'].mean()
initial_humidity = initial_conditions['Humidity'].mean()
initial_co2 = initial_conditions['CO2_room'].mean()
initial_satisfaction = initial_conditions['Satisfaction'].mean()

print(initial_temp, initial_humidity, initial_co2, initial_satisfaction)

19.00496896551724 46.096741379310345 220.8900689655172 95.54327473862068


In [7]:
# Set the initial state of the simulation
simulation_df.iloc[0] = {
    'Indoor Temperature': initial_temp,
    'Indoor Humidity': initial_humidity,
    'CO2 Levels': initial_co2,
    'Predicted Satisfaction': initial_satisfaction
}

print(simulation_df.head())

                    Indoor Temperature Indoor Humidity  CO2 Levels  \
2012-03-13 11:45:00          19.004969       46.096741  220.890069   
2012-03-13 12:00:00                NaN             NaN         NaN   
2012-03-13 12:15:00                NaN             NaN         NaN   
2012-03-13 12:30:00                NaN             NaN         NaN   
2012-03-13 12:45:00                NaN             NaN         NaN   

                    Predicted Satisfaction  
2012-03-13 11:45:00              95.543275  
2012-03-13 12:00:00                    NaN  
2012-03-13 12:15:00                    NaN  
2012-03-13 12:30:00                    NaN  
2012-03-13 12:45:00                    NaN  


In [8]:
def apply_control_logic(current_conditions, model, scaler, heating_counter, 
                        time_since_heating, time_since_ventilation, outdoor_temp, outdoor_humidity):
    
    # Arrange the features in the same order and structure as during the training
    features_df = pd.DataFrame([[
        current_conditions['CO2 Levels'], 
        current_conditions['Indoor Humidity'],
        current_conditions['Indoor Temperature']
    ]], columns=['CO2_room', 'Relative_humidity_room', 'Indoor_temperature_room'])
    
    # Scale the features using the scaler
    scaled_features = scaler.transform(features_df)
    
    # Make a prediction of satisfaction using the saved model
    satisfaction = model.predict(scaled_features)[0]

    # Check if satisfaction is below threshold
    action_taken = False
    if satisfaction < SATISFACTION_THRESHOLD:
        if heating_counter < MAX_HEATING_PER_DAY and time_since_heating >= HEATING_DURATION:
            # Apply heating effect
            current_conditions['Indoor Temperature'] += 0.5
            heating_counter += 1
            time_since_heating = timedelta(0)  # Reset heating duration timer
            action_taken = True
            # print("Heating applied")
        elif time_since_ventilation >= VENTILATION_DURATION:
            # Apply ventilation effect
            current_conditions['Indoor Temperature'] = outdoor_temp
            current_conditions['Indoor Humidity'] = outdoor_humidity
            time_since_ventilation = timedelta(0)  # Reset ventilation duration timer
            action_taken = True
            # print("Ventilation applied")

    # Always update timers
    time_since_heating += timedelta(minutes=15)
    time_since_ventilation += timedelta(minutes=15)

    # If an action was taken, recalculate satisfaction
    if action_taken:
        features_after_action_df = pd.DataFrame([[
            current_conditions['CO2 Levels'], 
            current_conditions['Indoor Humidity'], 
            current_conditions['Indoor Temperature']
        ]], columns=['CO2_room', 'Relative_humidity_room', 'Indoor_temperature_room'])
        scaled_features_after_action = scaler.transform(features_after_action_df)
        satisfaction = model.predict(scaled_features_after_action)[0]

    # Update satisfaction in the current conditions
    current_conditions['Predicted Satisfaction'] = satisfaction

    return current_conditions, heating_counter, time_since_heating, time_since_ventilation

In [9]:
# Initialise variables for the simulation
heating_counter = 0
time_since_heating = timedelta(hours=4)  # Allow immediate heating if required
time_since_ventilation = timedelta(minutes=15)  # Allow immediate ventilation if required

# Run the simulation
for timestamp in simulation_df.index:
    
    # Set the CO2 levels to the constant defined
    if pd.isna(simulation_df.loc[timestamp, 'CO2 Levels']):
        simulation_df.loc[timestamp, 'CO2 Levels'] = CO2_CONSTANT
    
    # For the first timestamp, initialize the conditions
    if timestamp == simulation_df.index[0]:
        simulation_df.loc[timestamp] = {
            'Indoor Temperature': initial_temp,
            'Indoor Humidity': initial_humidity,
            'CO2 Levels': CO2_CONSTANT,
            'Predicted Satisfaction': initial_satisfaction
        }
        continue
    
    # Copy the current conditions from the previous timestamp
    current_conditions = simulation_df.loc[timestamp - pd.Timedelta(minutes=15)].copy()
    
    # Get outdoor conditions
    outdoor_temp, outdoor_humidity = get_outdoor_conditions(timestamp, training_df)
        
    # Apply control logic
    current_conditions, heating_counter, time_since_heating, time_since_ventilation = apply_control_logic(
        current_conditions,
        trained_model,
        scaler,
        heating_counter,
        time_since_heating,
        time_since_ventilation,
        outdoor_temp,
        outdoor_humidity
    )
    
    # Assign the updated conditions back to the simulation DataFrame
    simulation_df.loc[timestamp] = current_conditions
    
    
    # Reset the heating counter at midnight
    if timestamp.time() == datetime.strptime('00:00:00', '%H:%M:%S').time():
        heating_counter = 0

# Once the loop is complete, print the first few rows to verify the simulation
print(simulation_df.head())

# Save the simulation results
simulation_df.to_csv('../data/SIMULATION_output.csv', index_label='Datetime')

                    Indoor Temperature Indoor Humidity  CO2 Levels  \
2012-03-13 11:45:00          19.004969       46.096741  194.431998   
2012-03-13 12:00:00          19.504969       46.096741  194.431998   
2012-03-13 12:15:00          19.504483       46.019634  194.431998   
2012-03-13 12:30:00           19.70069        46.05301  194.431998   
2012-03-13 12:45:00          19.885172       46.178828  194.431998   

                    Predicted Satisfaction  
2012-03-13 11:45:00              95.543275  
2012-03-13 12:00:00              96.271107  
2012-03-13 12:15:00              96.263623  
2012-03-13 12:30:00               96.43072  
2012-03-13 12:45:00              96.456661  
